In [0]:
#From : https://github.com/pytorch/examples/blob/master/vae/main.py

from __future__ import print_function
import argparse
import torch
import torch.utils.data
from torch import nn, optim
from torch.nn import functional as F
from torchvision import datasets, transforms
from torchvision.utils import save_image

In [0]:
device = "cuda"
kwargs = {'num_workers': 1, 'pin_memory': True}
train_loader = torch.utils.data.DataLoader(
datasets.MNIST('/', train = True, download = True,
              transform = transforms.ToTensor()),
    batch_size = 128, shuffle = True, **kwargs
)

test_loader = torch.utils.data.DataLoader(
datasets.MNIST('/', train = False, download = True,
              transform = transforms.ToTensor()),
    batch_size = 128, shuffle = True, **kwargs
)

0it [00:00, ?it/s]

9920512it [00:01, 6300710.01it/s]                             


Extracting /MNIST/raw/train-images-idx3-ubyte.gz


32768it [00:00, 275355.37it/s]                           
0it [00:00, ?it/s]

Extracting /MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 4766188.82it/s]                           
8192it [00:00, 130415.31it/s]


Extracting /MNIST/raw/t10k-images-idx3-ubyte.gz
Extracting /MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!


#Model

In [0]:
class VAE(nn.Module):
    def __init__(self):
        super(VAE, self).__init__()
        
        self.fc1 = nn.Linear(784, 400)
        self.fc21 = nn.Linear(400, 20)
        self.fc22 = nn.Linear(400, 20)
        self.fc3 = nn.Linear(20, 400)
        self.fc4 = nn.Linear(400, 784)
        
    def encode(self, x):
        h1 = F.relu(self.fc1(x))
        return  self.fc21(h1), self.fc22(h1)
    
    def reparameterize(self, mu, logvar): #https://www.quora.com/What-is-the-reparameterization-trick-in-variational-autoencoders/answer/Daniel-L%C3%A9vy
        std = torch.exp(0.5*logvar)
        eps = torch.randn_like(std)
        return mu + eps*std
    
    def decode(self, z):
        h3 = F.relu(self.fc3(z))
        return torch.sigmoid(self.fc4(h3))
    def forward(self, x):
        mu, logvar = self.encode(x.view(-1, 784))
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar
      
    

In [0]:
model = VAE().to(device)
optimizer = optim.Adam(model.parameters(), lr = 1e-3)


In [0]:
# Reconstruction + KL divergence losses summed over all elements and batch
def loss_function(recon_x, x, mu, logvar):
    BCE = F.binary_cross_entropy(recon_x, x.view(-1, 784), reduction='sum')

    # see Appendix B from VAE paper:
    # Kingma and Welling. Auto-Encoding Variational Bayes. ICLR, 2014
    # https://arxiv.org/abs/1312.6114
    # 0.5 * sum(1 + log(sigma^2) - mu^2 - sigma^2)
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())

    return BCE + KLD

In [0]:
def train(epoch):
    model.train()
    train_loss = 0
    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()
        recon_batch, mu, logvar = model(data)
        loss = loss_function(recon_batch, data, mu, logvar)
        loss.backward()
        train_loss += loss.item()
        optimizer.step()
        if batch_idx % 20 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader),
                loss.item() / len(data)))
    print('====> Epoch: {} Average loss: {:.4f}'.format(
          epoch, train_loss / len(train_loader.dataset)))

In [0]:
batch_size = 128
def test(epoch):
    model.eval()
    test_loss = 0
    with torch.no_grad():
        for i, (data, _) in enumerate(test_loader):
            data = data.to(device)
            recon_batch, mu, logvar = model(data)
            test_loss += loss_function(recon_batch, data, mu, logvar).item()
            if i == 0:
                n = min(data.size(0), 8)
                comparison = torch.cat([data[:n],
                                      recon_batch.view(batch_size, 1, 28, 28)[:n]])
                save_image(comparison.cpu(),
                         'reconstruction_' + str(epoch) + '.png', nrow=n)

    test_loss /= len(test_loader.dataset)
    print('====> Test set loss: {:.4f}'.format(test_loss))

In [0]:
  epochs = 20
  for epoch in range(1, epochs + 1):
        train(epoch)
        test(epoch)
        with torch.no_grad():
            sample = torch.randn(64, 20).to(device)
            sample = model.decode(sample).cpu()
            save_image(sample.view(64, 1, 28, 28),
                       'sample_' + str(epoch) + '.png')

Train Epoch: 1 [0/60000 (0%)]	Loss: 549.444946
Train Epoch: 1 [2560/60000 (4%)]	Loss: 237.495834
Train Epoch: 1 [5120/60000 (9%)]	Loss: 215.466019
Train Epoch: 1 [7680/60000 (13%)]	Loss: 198.516693
Train Epoch: 1 [10240/60000 (17%)]	Loss: 195.461823
Train Epoch: 1 [12800/60000 (21%)]	Loss: 186.745728
Train Epoch: 1 [15360/60000 (26%)]	Loss: 170.829422
Train Epoch: 1 [17920/60000 (30%)]	Loss: 163.038956
Train Epoch: 1 [20480/60000 (34%)]	Loss: 166.517609
Train Epoch: 1 [23040/60000 (38%)]	Loss: 150.477417
Train Epoch: 1 [25600/60000 (43%)]	Loss: 149.881927
Train Epoch: 1 [28160/60000 (47%)]	Loss: 152.486679
Train Epoch: 1 [30720/60000 (51%)]	Loss: 152.520584
Train Epoch: 1 [33280/60000 (55%)]	Loss: 139.888123
Train Epoch: 1 [35840/60000 (60%)]	Loss: 145.824341
Train Epoch: 1 [38400/60000 (64%)]	Loss: 141.768066
Train Epoch: 1 [40960/60000 (68%)]	Loss: 142.069504
Train Epoch: 1 [43520/60000 (72%)]	Loss: 133.177368
Train Epoch: 1 [46080/60000 (77%)]	Loss: 130.572830
Train Epoch: 1 [48640/

italicized text